In [1]:
import torch
from torch import nn
from torchvision import models

import sys
sys.path.append('..')

# Figure out L1 Feature Loss Inconsistency

In [ ]:
m = models.vgg19(pretrained=True)
m = m.eval()

In [ ]:
input = torch.ones(1,3,224,224)

In [ ]:
gen = m(input.detach())
fake = m(input.detach())

In [ ]:
m.features[2:20]

In [ ]:
for c in m.children():
    print(c)

In [ ]:
for i in m.modules():
    print(i)

In [ ]:
m(input)

In [ ]:
m(input)

In [ ]:
l1_loss = torch.nn.L1Loss()

In [ ]:
l1_loss(gen, fake)

In [ ]:
gen

In [ ]:
fake

In [ ]:
mm = m.double()
gengen = mm(input.double())
fakefake = mm(input.double())

In [ ]:
l1_loss.double()(gengen, fakefake)

# Examine model attributes

In [ ]:
features = list(models.vgg19(pretrained=True).features)[:23]
features

In [ ]:
# features的第3，8，15，22层分别是: relu1_2,relu2_2,relu3_3,relu4_3
features = nn.ModuleList(features).eval()
features

In [ ]:
len(features)

# Extract features from intermediate layers

Based on https://github.com/chenyuntc/pytorch-book/blob/master/chapter8-%E9%A3%8E%E6%A0%BC%E8%BF%81%E7%A7%BB(Neural%20Style)/PackedVGG.py

In [ ]:
# from abc import ABC
# class FeatureLoss(ABC, nn.Module):
#     def __init__(self, feature_extractor):
#         super().__init__()
#         # set to eval mode to disable dropout and such
#         self.feature_extractor = feature_extractor.eval()


In [3]:

# class MultiLayerFeatureLoss(FeatureLoss):
#     def __init__(self, feature_extractor, loss_fn=nn.L1Loss(), num_layers=3):
#         # e.g. VGG
#         super().__init__(feature_extractor)

#         features = list(feature_extractor.features)
#         self.num_layers = num_layers
#         self.loss_fn = loss_fn

#         self.layer_weights = [i + 1 / num_layers for i in range(num_layers)]

#         self.features = nn.ModuleList(features).eval()

#         start = len(self.features) - num_layers
#         end = len(self.features)
#         self.layers_to_keep = {i for i in range(start, end)}

#     def extract_intermediate_layers(self, x):
#         results = []
#         for ii, model in enumerate(self.features):
#             x = model(x)
#             if ii in self.layers_to_keep:
#                 results.append(x)

#         return results

#     def forward(self, generated, actual):
#         generated_feat_list = self.extract_intermediate_layers(generated)
#         actual_feat_list = self.extract_intermediate_layers(actual)
#         total_loss = 0

#         for i, w in enumerate(self.layer_weights):
#             total_loss += w * self.loss_fn(generated_feat_list[i], actual_feat_list[i])

#         return total_loss
    
from src.loss import MultiLayerFeatureLoss


In [ ]:
x = torch.ones(1,3,224,224).detach()

In [4]:
mll = MultiLayerFeatureLoss(models.vgg19(pretrained=True), num_layers=3)

In [ ]:
res1, res2 = mll.extract_intermediate_layers(x), mll.extract_intermediate_layers(x)

In [ ]:
res1

In [ ]:
mll(x, x)

# Try with my data

In [5]:
import sys
sys.path.append('..')

texture_dir = "../data/andrew/texture"
cloth_dir = "../data/andrew/clothing"
rois_db = "../data/andrew/rois.csv"

In [6]:
import os
os.listdir("../data")

['andrew']

In [7]:
from src.datasets import TextureDataset
texture_dataset = TextureDataset(
    texture_dir, rois_db, cloth_dir
)

In [10]:
from torch.utils.data import DataLoader

loader = DataLoader(texture_dataset, batch_size=1)

In [15]:
iterator = iter(loader)
x = next(iterator)
y = next(iterator)

In [16]:
mll(x[0], y[0])

tensor(0.1201, grad_fn=<AddBackward0>)